In [1]:
import pandas as pd

In [2]:
def concat_file(files):
    lfs = []
    for path in files:
        lfs.append(pd.read_csv("Data/"+path, skipinitialspace=True))
    return pd.concat(lfs)

In [77]:
listingsFiles = ["barcelona_listings.csv","berlin_listings_filtered.csv","madrid_listings_filtered.csv"]
listings = concat_file(listingsFiles).rename(columns={
    'host_id':'user_id',
    'host_name':'user_name'}).set_index('id')


In [4]:
def find_different_objects(listing, attribute, selected = False):
    values = listing[attribute]
    values = values.drop_duplicates()
    size = values.shape[0]
    if (selected or size <= 30):
        print(values)
    return values.shape[0]
    

In [5]:
tmp = []
for attribute in list(listings.columns.values):
    tmp.append(attribute +":"+ str(find_different_objects(listings, attribute)))

id
18666           within an hour
26033978          within a day
32711                      NaN
36763       within a few hours
7047052     a few days or more
Name: host_response_time, dtype: object
id
18666      ES
2137277    DE
Name: country_code, dtype: object
id
18666        Spain
2137277    Germany
Name: country, dtype: object
id
18666      Entire home/apt
21605         Private room
8521278        Shared room
Name: room_type, dtype: object
id
18666        6
18674        8
21605        2
31380        3
31958        4
32471        5
34981       10
35388        1
50066        9
67065        7
73683       12
474387      11
261750      13
478616      16
1098894     14
16938522    15
Name: accommodates, dtype: int64
id
18666        1.0
18674        2.0
32471        1.5
34981        3.0
71520        4.0
183858       NaN
186221       0.0
256982       0.5
267816       4.5
425639       3.5
478616      16.0
544995       2.5
1098894      6.0
1305640      5.0
4200509      7.5
5056503      5.5
6

In [6]:
print(tmp)

['listing_url:42094', 'name:40957', 'summary:38085', 'space:27385', 'description:40245', 'neighborhood_overview:21414', 'notes:14626', 'transit:22125', 'access:18965', 'interaction:17368', 'house_rules:20222', 'picture_url:41820', 'user_id:26659', 'host_url:26659', 'user_name:8291', 'host_since:2986', 'host_about:13783', 'host_response_time:5', 'host_response_rate:70', 'host_thumbnail_url:26590', 'host_picture_url:26590', 'host_neighbourhood:222', 'host_verifications:378', 'neighbourhood:224', 'city:74', 'country_code:2', 'country:2', 'latitude:42094', 'longitude:42094', 'property_type:37', 'room_type:3', 'accommodates:16', 'bathrooms:22', 'bedrooms:16', 'beds:28', 'bed_type:5', 'amenities:37765', 'square_feet:162', 'price:447', 'weekly_price:551', 'monthly_price:685', 'security_deposit:223', 'cleaning_fee:156', 'guests_included:16', 'extra_people:82', 'minimum_nights:112', 'maximum_nights:311', 'review_scores_rating:52', 'review_scores_accuracy:10', 'review_scores_cleanliness:10', 're

In [61]:
def get_id(attribute):
    
    words = attribute.split('_')
    letters = [word[0] for word in words]
    return ("".join(letters).lower())+'id'

In [107]:
def create_csv(data_frame, file_name, rules = {}):
    for  rule,attributes in rules.items():
        for attribute in attributes:
            data_frame[attribute] = data_frame[attribute].map(rule)
    data_frame.to_csv("Csv_to_database/"+file_name, line_terminator='\r')
def find_normal_values(dataframe, attribute=''):
    if attribute != '':
        df = dataframe[attribute]
    else:
        df = dataframe
    df = df.drop_duplicates()
    df = df.dropna()
    df.index = range(len(df))
    values = df.to_dict()
    values = {v: k for k, v in values.items()}
    df = df.to_frame()
    if attribute != '':
        df = df.rename_axis(get_id(attribute))
    return values, df
def normalize_value(val, normalized_values):
    if (val == ''):
        return val
    return normalized_values[val]
def create_normalized_value(dataframe, attribute):
    normalized_values, normal_entity = find_normal_values(dataframe, attribute)
    create_csv(normal_entity, attribute+".csv")
    return dataframe[attribute].apply(normalize_value, normalized_values = normalized_values)  

In [91]:
def create_list(df, attribute):
    lst = df[attribute].apply(lambda x: x[1:-1].replace('"',"").replace("'","").split(','))
    lst = lst.apply(lambda x : [y.strip(' ') for y in x])
    lst = lst.values.flatten()
    lst = [item for lst in lst for item in lst]
    lst = list(dict.fromkeys(lst))
    lst = list(filter(None, lst))
    lst = pd.DataFrame(lst,columns=[attribute])
    lst = pd.DataFrame(lst,columns=[attribute])
    lst = lst.rename_axis(get_id(attribute))
    lst = lst.drop_duplicates()
    return lst

In [92]:
amenities = create_list(listings,'amenities')
host_verifications = create_list(listings,'host_verifications')
host_verifications.head

<bound method NDFrame.head of          host_verifications
hvid                       
0                     email
1                     phone
2                   reviews
3                     jumio
4             government_id
5     offline_government_id
6                    selfie
7           identity_manual
8                  facebook
9                   sent_id
10               work_email
11           manual_offline
12                   google
13            manual_online
14             photographer
15                      kba
16                    weibo
17             zhima_selfie
18                   sesame
19           sesame_offline>

In [152]:
def create_list(df, attribute):
    lst = df[attribute].apply(lambda x: x[1:-1].replace('"',"").replace("'","").split(','))
    lst = lst.apply(lambda x : [y.strip(' ') for y in x])
    dct = lst.to_dict()
    tuples = []
    for key, values in dct.items():
        for value in values:
            tuples.append((key,value))

    dft = pd.DataFrame(tuples, columns=[get_id(attribute)])
    dft = dft.set_index('listing_id')
    dft = create_normalized_value(dft, attribute)
    
    dft = dft.to_frame()

    dft.to_csv("Csv_to_database/has_"+attribute+".csv")
    return dft

In [153]:
create_list(listings,'amenities')
create_list(listings,'host_verifications').head

    

<bound method NDFrame.head of          host_verifications
hvid                       
18666                     0
18666                     1
18666                     2
18666                     3
18666                     4
18674                     0
18674                     1
18674                     2
18674                     3
18674                     4
21605                     0
21605                     1
21605                     2
21605                     3
21605                     5
21605                     4
25786                     0
25786                     1
25786                     2
25786                     3
25786                     5
25786                     6
25786                     4
25786                     7
31377                     0
31377                     1
31377                     2
31380                     0
31380                     1
31380                     2
...                     ...
29851979                  1
29852920          

In [ ]:
list3 = [item for x in tmp for item in x]

In [ ]:
len(list3)

In [ ]:
list3 = list(dict.fromkeys(list3))

In [ ]:
find_different_objects(listings, 'city', True)